# Epilepsy Data Processing Pipeline

## Libraries

In [1]:
# Epilepsy Data Processing Pipeline
import os
import re
import json
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
from tqdm import tqdm
import pywt
import warnings
warnings.filterwarnings('ignore')

# MNE Libraries
import mne
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.preprocessing import ICA, create_eog_epochs, create_ecg_epochs
from mne_connectivity import spectral_connectivity_epochs
from mne.time_frequency import psd_array_multitaper

# Scipy and Scikit-learn
from scipy import signal, stats
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler

## Load Meta Data

In [2]:
def load_metadata_from_json(data_dir='data'):
    """Load patient and seizure metadata from JSON files using Polars"""
    data_dir = Path(data_dir)
    all_seizures = []
    all_patients = []
    
    json_files = sorted(list(data_dir.glob('**/*.json')))
    print(f"Found {len(json_files)} JSON files")
    
    for json_file in json_files:
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            # Extract patient-level information
            patient_info = {
                'patient_id': data['patient_id'],
                'sampling_rate_hz': data['sampling_rate_hz'],
                'num_channels': len(data['channels']),
                'json_file_path': str(json_file)
            }
            
            if 'file_name' in data:
                patient_info['file_name'] = data['file_name']
                patient_info['registration_start_time'] = data.get('registration_start_time')
                patient_info['registration_end_time'] = data.get('registration_end_time')
            
            all_patients.append(patient_info)
            
            # Process each seizure
            for seizure in data['seizures']:
                seizure_record = {
                    'patient_id': data['patient_id'],
                    'sampling_rate_hz': data['sampling_rate_hz'],
                    'seizure_number': seizure['seizure_number']
                }
                
                for key, value in seizure.items():
                    seizure_record[key] = value
                
                if 'file_name' in patient_info and 'file_name' not in seizure:
                    seizure_record['file_name'] = patient_info['file_name']
                    if 'registration_start_time' in patient_info:
                        seizure_record['registration_start_time'] = patient_info['registration_start_time']
                        seizure_record['registration_end_time'] = patient_info['registration_end_time']
                
                all_seizures.append(seizure_record)
                
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
            continue
    
    # Convert to Polars DataFrames
    seizures_df = pl.DataFrame(all_seizures)
    patients_df = pl.DataFrame(all_patients)
    
    return seizures_df, patients_df

## Processing Functions

In [3]:
def extract_psd_features(raw, fmin=0.5, fmax=50):
    """Extract PSD features without restrictive conditions"""
    features = {}
    
    try:
        # Get data
        data = raw.get_data()
        sfreq = raw.info['sfreq']
        
        # Calculate PSD using multitaper (more robust than welch for non-stationary signals)
        psds, freqs = psd_array_multitaper(
            data, sfreq, fmin=fmin, fmax=fmax, 
            adaptive=True, normalization='full', verbose=False
        )
        
        # Extended frequency bands
        bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 12),
            'low_beta': (12, 20),
            'high_beta': (20, 30),
            'gamma': (30, 50),
            'high_gamma': (50, 100)
        }
        
        band_powers = {}
        
        for band_name, (low_freq, high_freq) in bands.items():
            freq_mask = (freqs >= low_freq) & (freqs < high_freq)
            if freq_mask.any():
                band_power = np.mean(psds[:, freq_mask], axis=1)
                band_powers[band_name] = band_power
                
                # Store band power statistics
                features[f'psd_{band_name}_mean'] = float(np.mean(band_power))
                features[f'psd_{band_name}_std'] = float(np.std(band_power))
                features[f'psd_{band_name}_cv'] = float(np.std(band_power) / (np.mean(band_power) + 1e-10))  # Coefficient of variation
        
        # Calculate band power ratios (important for seizure detection)
        if 'theta' in band_powers and 'alpha' in band_powers:
            features['psd_theta_alpha_ratio'] = float(np.mean(band_powers['theta']) / (np.mean(band_powers['alpha']) + 1e-10))
        
        if 'delta' in band_powers and 'alpha' in band_powers:
            features['psd_delta_alpha_ratio'] = float(np.mean(band_powers['delta']) / (np.mean(band_powers['alpha']) + 1e-10))
        
        if 'low_beta' in band_powers and 'high_beta' in band_powers:
            features['psd_beta_ratio'] = float(np.mean(band_powers['high_beta']) / (np.mean(band_powers['low_beta']) + 1e-10))
        
        # Spectral edge frequencies (SEF50, SEF90, SEF95)
        mean_psd = np.mean(psds, axis=0)
        cumsum_psd = np.cumsum(mean_psd)
        cumsum_psd = cumsum_psd / cumsum_psd[-1]
        
        for percentile in [50, 75, 90, 95]:
            edge_idx = np.where(cumsum_psd >= percentile/100)[0]
            if len(edge_idx) > 0:
                features[f'psd_sef{percentile}'] = float(freqs[edge_idx[0]])
            else:
                features[f'psd_sef{percentile}'] = float(freqs[-1])
        
        # Spectral centroid and spread
        freq_weights = freqs * mean_psd
        spectral_centroid = np.sum(freq_weights) / (np.sum(mean_psd) + 1e-10)
        features['psd_spectral_centroid'] = float(spectral_centroid)
        
        # Spectral spread (standard deviation around centroid)
        spectral_spread = np.sqrt(np.sum(((freqs - spectral_centroid) ** 2) * mean_psd) / (np.sum(mean_psd) + 1e-10))
        features['psd_spectral_spread'] = float(spectral_spread)
        
        # Spectral skewness and kurtosis
        if spectral_spread > 0:
            spectral_skewness = np.sum(((freqs - spectral_centroid) ** 3) * mean_psd) / ((spectral_spread ** 3) * np.sum(mean_psd) + 1e-10)
            spectral_kurtosis = np.sum(((freqs - spectral_centroid) ** 4) * mean_psd) / ((spectral_spread ** 4) * np.sum(mean_psd) + 1e-10)
            features['psd_spectral_skewness'] = float(spectral_skewness)
            features['psd_spectral_kurtosis'] = float(spectral_kurtosis)
        else:
            features['psd_spectral_skewness'] = 0.0
            features['psd_spectral_kurtosis'] = 0.0
        
    except Exception as e:
        print(f"Error in enhanced PSD calculation: {e}")
    
    return features

In [4]:
def extract_psd_features_by_region(raw, fmin=0.5, fmax=50):
    # Define channel groups
    channel_groups = {
        'frontal': ['Fp1', 'Fp2', 'F3', 'F4', 'F7', 'F8', 'Fz'],
        'central': ['C3', 'C4', 'Cz'],
        'parietal': ['P3', 'P4', 'Pz'],
        'occipital': ['O1', 'O2'],
        'temporal': ['T3', 'T4', 'T5', 'T6']
    }
    
    regional_features = {}
    available_channels = raw.ch_names
    
    for region, channel_list in channel_groups.items():
        # Find matching channels (handle different naming conventions)
        region_channels = []
        for ch in available_channels:
            ch_clean = ch.upper().replace('EEG', '').replace('-', '').replace(' ', '').strip()
            for target_ch in channel_list:
                if target_ch.upper() in ch_clean or ch_clean in target_ch.upper():
                    region_channels.append(ch)
                    break
        
        try:
            # Pick channels for this region
            raw_region = raw.copy().pick(region_channels)
            
            # Extract features for this region
            region_psd_features = extract_psd_features(raw_region, fmin, fmax)
            
            # Add region prefix to feature names
            for feature_name, value in region_psd_features.items():
                regional_features[f'{region}_{feature_name}'] = value
                
        except Exception as e:
            print(f"Could not process {region} region: {e}")   
    
    return regional_features

In [5]:
def parse_seizure_time(time_str):
    if pd.isna(time_str) or time_str == '' or time_str is None:
        return None
    
    try:
        time_str = str(time_str).strip()
        
        # Handle HH:MM:SS or MM:SS format
        if '.' in time_str:
            parts = time_str.split('.')
            if len(parts) == 3:
                hours, minutes, seconds = map(float, parts)
                return hours * 3600 + minutes * 60 + seconds
            elif len(parts) == 2:
                minutes, seconds = map(float, parts)
                return minutes * 60 + seconds
        
        # Handle pure seconds
        return float(time_str)
        
    except Exception as e:
        print(f"Could not parse time '{time_str}': {e}")
        return None

### Differential Entropy

In [6]:
def calculate_differential_entropy(data, sfreq, bands=None):
    if bands is None:
        bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 12),
            'beta': (12, 30),
            'gamma': (30, 50),
            'high_gamma': (50, 100)  # Added high gamma
        }
    
    features = {}
    n_channels = data.shape[0]
    
    for band_name, (low_freq, high_freq) in bands.items():
        # Bandpass filter for each band
        try:
            filtered_data = mne.filter.filter_data(
                data, sfreq, l_freq=low_freq, h_freq=high_freq, 
                verbose=False, method='fir', fir_design='firwin'
            )
            
            # Calculate variance for each channel
            variances = np.var(filtered_data, axis=1)
            
            # Calculate DE: 0.5 * log(2 * pi * e * variance)
            # Adding small epsilon to avoid log(0)
            de_values = 0.5 * np.log(2 * np.pi * np.e * (variances + 1e-10))
            
            # Store statistics
            features[f'de_{band_name}_mean'] = float(np.mean(de_values))
            features[f'de_{band_name}_std'] = float(np.std(de_values))
            features[f'de_{band_name}_median'] = float(np.median(de_values))
            features[f'de_{band_name}_max'] = float(np.max(de_values))
            features[f'de_{band_name}_min'] = float(np.min(de_values))
            
            # Asymmetry features (frontal asymmetry is important for emotion/seizure)
            if n_channels >= 2:
                # Calculate hemispheric asymmetry
                left_channels = de_values[:n_channels//2]
                right_channels = de_values[n_channels//2:]
                min_len = min(len(left_channels), len(right_channels))
                if min_len > 0:
                    asymmetry = left_channels[:min_len] - right_channels[:min_len]
                    features[f'de_{band_name}_asymmetry_mean'] = float(np.mean(asymmetry))
                    features[f'de_{band_name}_asymmetry_std'] = float(np.std(asymmetry))
                
        except Exception as e:
            print(f"Error calculating DE for {band_name}: {e}")
    
    return features

### Propagation

In [7]:
def calculate_simple_propagation_features(raw, seizure_start=None, seizure_end=None):
    features = {}
    
    try:
        data = raw.get_data()
        sfreq = raw.info['sfreq']
        n_channels, n_samples = data.shape
        
        # Apply bandpass filter for seizure activity
        data_filtered = mne.filter.filter_data(
            data, sfreq, l_freq=3, h_freq=30, verbose=False
        )
        
        # Calculate channel-wise power changes
        onset_times = []
        
        for ch_idx in range(n_channels):
            channel_data = data_filtered[ch_idx, :]
            
            # Calculate envelope
            analytic_signal = signal.hilbert(channel_data)
            envelope = np.abs(analytic_signal)
            
            # Simple smoothing
            window_samples = max(3, int(0.1 * sfreq))
            if window_samples % 2 == 0:
                window_samples += 1
            envelope_smooth = signal.savgol_filter(
                envelope, 
                window_samples, 
                min(1, window_samples-1)
            )
            
            # Find the time of maximum activity
            if seizure_start is not None and seizure_end is not None:
                # Look in seizure window
                start_sample = max(0, int(seizure_start * sfreq))
                end_sample = min(n_samples, int(seizure_end * sfreq))
                if start_sample < end_sample:
                    seizure_segment = envelope_smooth[start_sample:end_sample]
                    if len(seizure_segment) > 0:
                        max_idx = np.argmax(seizure_segment) + start_sample
                        onset_times.append(max_idx / sfreq)
            else:
                # Use entire recording
                if len(envelope_smooth) > 0:
                    max_idx = np.argmax(envelope_smooth)
                    onset_times.append(max_idx / sfreq)
        
        # Only calculate propagation statistics if we have onset times
        if len(onset_times) > 0:
            # Calculate delays between channels
            onset_times = np.array(onset_times)
            sorted_onsets = np.sort(onset_times)
            delays = np.diff(sorted_onsets)
            
            # Simple propagation speed estimate (assuming 10cm average distance)
            avg_distance_mm = 100  # 100mm = 10cm
            speeds = []
            
            # Calculate speeds only for positive delays
            positive_delays = delays[delays > 0]
            if len(positive_delays) > 0:
                for delay in positive_delays:
                    speed = avg_distance_mm / delay
                    speeds.append(speed)
            
            # Calculate speed statistics only if we have speeds
            if len(speeds) > 0:
                features['mean_propagation_speed'] = float(np.mean(speeds))
                features['median_propagation_speed'] = float(np.median(speeds))
                features['std_propagation_speed'] = float(np.std(speeds))
                features['max_propagation_speed'] = float(np.max(speeds))
                features['min_propagation_speed'] = float(np.min(speeds))
                features['num_propagation_events'] = len(speeds)
            
            # Calculate delay statistics
            if len(delays) > 0:
                features['mean_onset_delay'] = float(np.mean(delays))
                features['max_onset_delay'] = float(np.max(delays))
                
    except Exception as e:
        print(f"Error in propagation calculation: {e}")
    
    return features

### Wavelet

In [8]:
def calculate_wavelet_features(data, sfreq, wavelet='db4', levels=5):
    features = {}
    n_channels = data.shape[0]
    
    # Store features for each decomposition level
    all_channel_features = []
    
    for ch_idx in range(n_channels):
        channel_data = data[ch_idx, :]
        
        try:
            # Perform wavelet decomposition
            coeffs = pywt.wavedec(channel_data, wavelet, level=levels)
            
            # Calculate features for each level
            channel_features = []
            for level, coeff in enumerate(coeffs):
                if len(coeff) > 0:
                    # Energy of coefficients
                    energy = np.sum(coeff ** 2)
                    # Entropy
                    entropy = stats.entropy(np.abs(coeff) + 1e-10)
                    # Statistical features
                    mean_coeff = np.mean(np.abs(coeff))
                    std_coeff = np.std(coeff)
                    max_coeff = np.max(np.abs(coeff))
                    
                    channel_features.extend([energy, entropy, mean_coeff, std_coeff, max_coeff])
                else:
                    channel_features.extend([0.0, 0.0, 0.0, 0.0, 0.0])
            
            all_channel_features.append(channel_features)
            
        except Exception as e:
            print(f"Error in wavelet transform for channel {ch_idx}: {e}")
            # Add zeros for failed channel
            all_channel_features.append([0.0] * (5 * (levels + 1)))
    
    # Aggregate across channels
    all_channel_features = np.array(all_channel_features)
    
    # Store aggregated features
    feature_names = ['energy', 'entropy', 'mean', 'std', 'max']
    for level in range(levels + 1):
        for feat_idx, feat_name in enumerate(feature_names):
            feat_values = all_channel_features[:, level * 5 + feat_idx]
            features[f'wt_level{level}_{feat_name}_mean'] = float(np.mean(feat_values))
            features[f'wt_level{level}_{feat_name}_std'] = float(np.std(feat_values))
    
    # Calculate wavelet packet energy distribution
    try:
        # Use first channel for packet decomposition (computationally expensive for all)
        wp = pywt.WaveletPacket(data[0, :], wavelet, maxlevel=3)
        packet_energies = []
        for node in wp.get_level(3):
            packet_energies.append(np.sum(node.data ** 2))
        
        if packet_energies:
            total_energy = sum(packet_energies)
            if total_energy > 0:
                normalized_energies = np.array([e/total_energy for e in packet_energies])
                features['wt_packet_entropy'] = float(stats.entropy(normalized_energies + 1e-10))
            else:
                features['wt_packet_entropy'] = 0.0
        else:
            features['wt_packet_entropy'] = 0.0
            
    except Exception as e:
        print(f"Error in wavelet packet decomposition: {e}")
    
    return features

### Time Domain

In [9]:
def calculate_time_domain_features(data, sfreq):
    features = {}
    n_channels, n_samples = data.shape
    
    try:
        # Statistical features for each channel
        channel_features = {
            'mean': np.mean(data, axis=1),
            'std': np.std(data, axis=1),
            'skewness': stats.skew(data, axis=1),
            'kurtosis': stats.kurtosis(data, axis=1),
            'rms': np.sqrt(np.mean(data ** 2, axis=1)),
            'peak_to_peak': np.ptp(data, axis=1),
            'zero_crossings': np.sum(np.diff(np.sign(data), axis=1) != 0, axis=1) / (n_samples / sfreq),  # Rate per second
        }
        
        # Hjorth parameters
        # Activity (variance of signal)
        activity = np.var(data, axis=1)
        
        # Mobility (standard deviation of first derivative / standard deviation of signal)
        first_deriv = np.diff(data, axis=1)
        mobility = np.std(first_deriv, axis=1) / (np.std(data, axis=1) + 1e-10)
        
        # Complexity (mobility of first derivative / mobility of signal)
        second_deriv = np.diff(first_deriv, axis=1)
        mobility_deriv = np.std(second_deriv, axis=1) / (np.std(first_deriv, axis=1) + 1e-10)
        complexity = mobility_deriv / (mobility + 1e-10)
        
        channel_features['hjorth_activity'] = activity
        channel_features['hjorth_mobility'] = mobility
        channel_features['hjorth_complexity'] = complexity
        
        # Line length (sum of absolute differences)
        line_length = np.sum(np.abs(np.diff(data, axis=1)), axis=1) / n_samples
        channel_features['line_length'] = line_length
        
        # Non-linear energy
        nonlinear_energy = []
        for ch in range(n_channels):
            if n_samples >= 3:
                nle = np.mean(data[ch, 1:-1]**2 - data[ch, :-2] * data[ch, 2:])
                nonlinear_energy.append(nle)
            else:
                nonlinear_energy.append(0.0)
        channel_features['nonlinear_energy'] = np.array(nonlinear_energy)
        
        # Aggregate features across channels
        for feat_name, feat_values in channel_features.items():
            features[f'time_{feat_name}_mean'] = float(np.mean(feat_values))
            features[f'time_{feat_name}_std'] = float(np.std(feat_values))
            features[f'time_{feat_name}_max'] = float(np.max(feat_values))
            features[f'time_{feat_name}_min'] = float(np.min(feat_values))
        
    except Exception as e:
        print(f"Error in time-domain feature calculation: {e}")
    
    return features

### Connectivity

In [10]:
def calculate_connectivity_features(raw, fmin=1, fmax=50):
    features = {}
    
    try:
        data = raw.get_data()
        n_channels, n_samples = data.shape
        sfreq = raw.info['sfreq']
        
        # Only calculate if we have multiple channels
        if n_channels < 2:
            raise ValueError("Need at least 2 channels for connectivity")
        
        # Reshape data for connectivity calculation (n_epochs, n_channels, n_times)
        # Create pseudo-epochs by segmenting the data
        epoch_length = int(2 * sfreq)  # 2-second epochs
        n_epochs = n_samples // epoch_length
        
        if n_epochs > 0:
            epochs_data = []
            for i in range(n_epochs):
                start = i * epoch_length
                end = start + epoch_length
                epochs_data.append(data[:, start:end])
            epochs_data = np.array(epochs_data)
            
            # Calculate spectral connectivity
            # Using coherence as the connectivity measure
            con = spectral_connectivity_epochs(
                epochs_data, method='coh', mode='multitaper',
                sfreq=sfreq, fmin=fmin, fmax=fmax,
                verbose=False
            )
            
            # Get connectivity matrix (n_channels x n_channels x n_freqs)
            con_matrix = con.get_data(output='dense')
            
            # Average across frequencies
            mean_connectivity = np.mean(con_matrix, axis=2)
            
            # Extract upper triangle (excluding diagonal)
            upper_tri = np.triu_indices(n_channels, k=1)
            connectivity_values = mean_connectivity[upper_tri]
            
            # Store connectivity statistics
            features['connectivity_mean'] = float(np.mean(connectivity_values))
            features['connectivity_std'] = float(np.std(connectivity_values))
            features['connectivity_max'] = float(np.max(connectivity_values))
            features['connectivity_min'] = float(np.min(connectivity_values))
            
            # Global efficiency (mean of connectivity)
            features['global_efficiency'] = float(np.mean(connectivity_values))
            
            # Node strength (sum of connections for each node)
            node_strengths = np.sum(mean_connectivity, axis=0) - 1  # Subtract diagonal
            features['node_strength_mean'] = float(np.mean(node_strengths))
            features['node_strength_std'] = float(np.std(node_strengths))
            features['node_strength_max'] = float(np.max(node_strengths))
            
            # Clustering coefficient (simplified version)
            clustering_coeffs = []
            for i in range(n_channels):
                neighbors = mean_connectivity[i, :] > 0.3  # Threshold for connection
                n_neighbors = np.sum(neighbors) - 1  # Exclude self
                if n_neighbors > 1:
                    # Count connections between neighbors
                    neighbor_indices = np.where(neighbors)[0]
                    neighbor_connections = 0
                    for j in range(len(neighbor_indices)):
                        for k in range(j+1, len(neighbor_indices)):
                            if mean_connectivity[neighbor_indices[j], neighbor_indices[k]] > 0.3:
                                neighbor_connections += 1
                    max_connections = n_neighbors * (n_neighbors - 1) / 2
                    if max_connections > 0:
                        clustering = neighbor_connections / max_connections
                        clustering_coeffs.append(clustering)
            
            features['clustering_coefficient'] = float(np.mean(clustering_coeffs))
  
        else:
            raise ValueError("Not enough data for connectivity analysis")
            
    except Exception as e:
        print(f"Error in connectivity calculation: {e}")
    
    return features

### PAC

In [11]:
def calculate_pac_features(data, sfreq):
    features = {}
    
    try:
        n_channels = data.shape[0]
        
        # Define phase and amplitude frequency bands
        phase_bands = {
            'theta': (4, 8),
            'alpha': (8, 12)
        }
        
        amplitude_bands = {
            'gamma': (30, 50),
            'high_gamma': (50, 100)
        }
        
        pac_values = []
        
        for phase_name, (phase_low, phase_high) in phase_bands.items():
            for amp_name, (amp_low, amp_high) in amplitude_bands.items():
                
                channel_pac = []
                for ch in range(min(n_channels, 10)):  # Limit to first 10 channels for speed
                    # Extract phase
                    phase_filtered = mne.filter.filter_data(
                        data[ch:ch+1, :], sfreq, 
                        l_freq=phase_low, h_freq=phase_high,
                        verbose=False
                    )
                    phase = np.angle(signal.hilbert(phase_filtered[0]))
                    
                    # Extract amplitude
                    amp_filtered = mne.filter.filter_data(
                        data[ch:ch+1, :], sfreq,
                        l_freq=amp_low, h_freq=amp_high,
                        verbose=False
                    )
                    amplitude = np.abs(signal.hilbert(amp_filtered[0]))
                    
                    # Calculate PAC using Modulation Index
                    n_bins = 18
                    phase_bins = np.linspace(-np.pi, np.pi, n_bins + 1)
                    amp_by_phase = []
                    
                    for i in range(n_bins):
                        mask = (phase >= phase_bins[i]) & (phase < phase_bins[i+1])
                        if np.sum(mask) > 0:
                            amp_by_phase.append(np.mean(amplitude[mask]))
                        else:
                            amp_by_phase.append(0)
                    
                    # Normalize and calculate entropy
                    amp_by_phase = np.array(amp_by_phase)
                    if np.sum(amp_by_phase) > 0:
                        amp_by_phase = amp_by_phase / np.sum(amp_by_phase)
                        # Kullback-Leibler divergence from uniform distribution
                        uniform = np.ones(n_bins) / n_bins
                        kl_div = np.sum(amp_by_phase * np.log((amp_by_phase + 1e-10) / uniform))
                        mi = kl_div / np.log(n_bins)  # Normalized MI
                        channel_pac.append(mi)
                    else:
                        channel_pac.append(0.0)
                
                if channel_pac:
                    pac_value = np.mean(channel_pac)
                    features[f'pac_{phase_name}_{amp_name}'] = float(pac_value)
                    pac_values.append(pac_value)
                else:
                    features[f'pac_{phase_name}_{amp_name}'] = 0.0
        
        # Overall PAC statistics
        features['pac_mean'] = float(np.mean(pac_values))
        features['pac_max'] = float(np.max(pac_values))
            
    except Exception as e:
        print(f"Error in PAC calculation: {e}")
    
    return features

### Sample, Permutation, and Approximate Entropy

In [12]:
def calculate_entropy_features(data, sfreq):
    features = {}
    
    try:
        n_channels = data.shape[0]
        
        # Sample Entropy
        sample_entropies = []
        for ch in range(min(n_channels, 10)):  # Limit channels for speed
            # Downsample for faster computation
            downsampled = signal.resample(data[ch, :], min(1000, len(data[ch, :])))
            # Simple sample entropy approximation
            se = calculate_sample_entropy(downsampled, m=2, r=0.2*np.std(downsampled))
            sample_entropies.append(se)
        
        features['sample_entropy_mean'] = float(np.mean(sample_entropies))
        features['sample_entropy_std'] = float(np.std(sample_entropies))
        
        # Permutation Entropy
        perm_entropies = []
        for ch in range(min(n_channels, 10)):
            pe = calculate_permutation_entropy(data[ch, :], order=3, delay=1)
            perm_entropies.append(pe)
        
        features['permutation_entropy_mean'] = float(np.mean(perm_entropies))
        features['permutation_entropy_std'] = float(np.std(perm_entropies))
        
        # Approximate Entropy (simplified)
        approx_entropies = []
        for ch in range(min(n_channels, 10)):
            downsampled = signal.resample(data[ch, :], min(1000, len(data[ch, :])))
            ae = calculate_approx_entropy(downsampled, m=2, r=0.2*np.std(downsampled))
            approx_entropies.append(ae)
        
        features['approx_entropy_mean'] = float(np.mean(approx_entropies))
        features['approx_entropy_std'] = float(np.std(approx_entropies))
        
    except Exception as e:
        print(f"Error in entropy calculation: {e}")
        features['sample_entropy_mean'] = 0.0
        features['sample_entropy_std'] = 0.0
        features['permutation_entropy_mean'] = 0.0
        features['permutation_entropy_std'] = 0.0
        features['approx_entropy_mean'] = 0.0
        features['approx_entropy_std'] = 0.0
    
    return features

def calculate_sample_entropy(signal_data, m=2, r=0.2):
    """Calculate sample entropy"""
    N = len(signal_data)
    
    def _maxdist(x_i, x_j, m):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
    
    def _phi(m):
        patterns = np.array([signal_data[i:i+m] for i in range(N - m + 1)])
        C = 0
        for i in range(len(patterns)):
            for j in range(i+1, len(patterns)):
                if _maxdist(patterns[i], patterns[j], m) <= r:
                    C += 1
        return C
    
    try:
        phi_m = _phi(m)
        phi_m1 = _phi(m + 1)
        
        if phi_m == 0:
            return 0
        return -np.log(phi_m1 / phi_m) if phi_m1 > 0 else 0
    except:
        return 0

def calculate_permutation_entropy(signal_data, order=3, delay=1):
    """Calculate permutation entropy"""
    try:
        n = len(signal_data)
        permutations = []
        
        for i in range(n - delay * (order - 1)):
            indices = [i + j * delay for j in range(order)]
            sorted_indices = np.argsort(signal_data[indices])
            permutations.append(tuple(sorted_indices))
        
        # Count occurrences
        unique, counts = np.unique(permutations, axis=0, return_counts=True)
        probs = counts / len(permutations)
        
        # Calculate entropy
        return -np.sum(probs * np.log(probs + 1e-10))
    except:
        return 0

def calculate_approx_entropy(signal_data, m=2, r=0.2):
    """Calculate approximate entropy"""
    try:
        N = len(signal_data)
        
        def _maxdist(x_i, x_j, m):
            return max([abs(ua - va) for ua, va in zip(x_i, x_j)])
        
        def _phi(m):
            patterns = np.array([signal_data[i:i+m] for i in range(N - m + 1)])
            C = np.zeros(N - m + 1)
            
            for i in range(N - m + 1):
                matches = 0
                for j in range(N - m + 1):
                    if _maxdist(patterns[i], patterns[j], m) <= r:
                        matches += 1
                C[i] = matches / (N - m + 1)
            
            return np.mean(np.log(C + 1e-10))
        
        return _phi(m) - _phi(m + 1)
    except:
        return 0

## Feature Extraction Function

In [13]:
def extract_comprehensive_features(raw, seizure_start=None, seizure_end=None):
    all_features = {}
    
    # Get data
    data = raw.get_data()
    sfreq = raw.info['sfreq']
    
    de_features = calculate_differential_entropy(data, sfreq)
    all_features.update(de_features)
    
    psd_features = extract_psd_features(raw)
    all_features.update(psd_features)
    
    wt_features = calculate_wavelet_features(data, sfreq)
    all_features.update(wt_features)
    
    time_features = calculate_time_domain_features(data, sfreq)
    all_features.update(time_features)
    
    connectivity_features = calculate_connectivity_features(raw)
    all_features.update(connectivity_features)
    
    pac_features = calculate_pac_features(data, sfreq)
    all_features.update(pac_features)
    
    entropy_features = calculate_entropy_features(data, sfreq)
    all_features.update(entropy_features)
    
    # Add seizure-specific features if seizure times are provided
    if seizure_start is not None and seizure_end is not None and seizure_start > 0 and seizure_end > 0:
        # Extract pre-ictal, ictal, and post-ictal features
        pre_ictal_start = max(0, seizure_start - 30)  # 30 seconds before
        pre_ictal_end = seizure_start
        
        post_ictal_start = seizure_end
        post_ictal_end = min(data.shape[1] / sfreq, seizure_end + 30)  # 30 seconds after
        
        # Extract features for different periods
        try:
            # Pre-ictal
            pre_start_sample = int(pre_ictal_start * sfreq)
            pre_end_sample = int(pre_ictal_end * sfreq)
            if pre_end_sample > pre_start_sample:
                pre_data = data[:, pre_start_sample:pre_end_sample]
                pre_de = calculate_differential_entropy(pre_data, sfreq)
                for key, value in pre_de.items():
                    all_features[f'preictal_{key}'] = value
            
            # Ictal
            ictal_start_sample = int(seizure_start * sfreq)
            ictal_end_sample = int(seizure_end * sfreq)
            if ictal_end_sample > ictal_start_sample:
                ictal_data = data[:, ictal_start_sample:ictal_end_sample]
                ictal_de = calculate_differential_entropy(ictal_data, sfreq)
                for key, value in ictal_de.items():
                    all_features[f'ictal_{key}'] = value
            
            # Post-ictal
            post_start_sample = int(post_ictal_start * sfreq)
            post_end_sample = int(post_ictal_end * sfreq)
            if post_end_sample > post_start_sample:
                post_data = data[:, post_start_sample:post_end_sample]
                post_de = calculate_differential_entropy(post_data, sfreq)
                for key, value in post_de.items():
                    all_features[f'postictal_{key}'] = value
                    
        except Exception as e:
            print(f"Error extracting seizure period features: {e}")
    
    return all_features

## Process Single EDF File Function

In [14]:
def process_single_edf(edf_path, seizure_info):
    features = {}
    
    try:
        # Load EDF file
        raw = mne.io.read_raw_edf(str(edf_path), preload=True, verbose=False)
        
        # Basic file info
        features['file_path'] = str(edf_path)
        features['num_channels'] = len(raw.ch_names)
        features['sampling_rate'] = raw.info['sfreq']
        features['duration_seconds'] = raw.n_times / raw.info['sfreq']
        
        # Apply bandpass filter
        raw.filter(0.5, 100, fir_design='firwin', verbose=False)
        
        # Parse seizure times
        seizure_start = None
        seizure_end = None
        
        if isinstance(seizure_info, (dict, pd.Series)):
            if 'seizure_start_time' in seizure_info:
                time_str = str(seizure_info.get('seizure_start_time')).strip()
                if ':' in time_str or '.' in time_str:
                    time_str = time_str.replace('.', ':')
                    parts = time_str.split(':')
                    if len(parts) == 3:
                        hours, minutes, seconds = map(float, parts)
                        seizure_start = hours * 3600 + minutes * 60 + seconds
                    elif len(parts) == 2:
                        minutes, seconds = map(float, parts)
                        seizure_start = minutes * 60 + seconds
                else:
                    try:
                        seizure_start = float(time_str)
                    except:
                        seizure_start = None
            
            if 'seizure_end_time' in seizure_info:
                time_str = str(seizure_info.get('seizure_end_time')).strip()
                if ':' in time_str or '.' in time_str:
                    time_str = time_str.replace('.', ':')
                    parts = time_str.split(':')
                    if len(parts) == 3:
                        hours, minutes, seconds = map(float, parts)
                        seizure_end = hours * 3600 + minutes * 60 + seconds
                    elif len(parts) == 2:
                        minutes, seconds = map(float, parts)
                        seizure_end = minutes * 60 + seconds
                else:
                    try:
                        seizure_end = float(time_str)
                    except:
                        seizure_end = None
        
        # Extract comprehensive features
        comprehensive_features = extract_comprehensive_features(raw, seizure_start, seizure_end)
        features.update(comprehensive_features)
        
        # Propagation features
        propagation_features = calculate_simple_propagation_features(raw, seizure_start, seizure_end)
        features.update(propagation_features)
        
        # Add seizure timing info
        features['seizure_start_seconds'] = seizure_start if seizure_start is not None else 0.0
        features['seizure_end_seconds'] = seizure_end if seizure_end is not None else 0.0
        if seizure_end and seizure_start:
            features['seizure_duration'] = seizure_end - seizure_start
        else:
            features['seizure_duration'] = 0.0
        
        features['processing_success'] = True
        features['error_message'] = ''
        
    except Exception as e:
        print(f"Error processing {edf_path}: {e}")
        features['processing_success'] = False
        features['error_message'] = str(e)
    
    return features

## Build Comprehensive Dataset

In [15]:
def build_comprehensive_dataset(seizures_df, patients_df, 
                              data_root_paths=['data/siena_scalp', 'data/chb-mit']):
    all_records = []
    
    # Convert Polars to pandas for iteration (if needed)
    if isinstance(seizures_df, pl.DataFrame):
        seizures_pd = seizures_df.to_pandas()
        patients_pd = patients_df.to_pandas()
    else:
        seizures_pd = seizures_df
        patients_pd = patients_df
    
    print(f"Processing {len(seizures_pd)} seizure records...")
    print("="*60)
    
    for idx, seizure_row in tqdm(seizures_pd.iterrows(), total=len(seizures_pd)):
        record = {}
        
        # Add all seizure metadata
        for col in seizure_row.index:
            record[f'seizure_{col}'] = seizure_row[col]
        
        # Find corresponding patient info
        patient_id = seizure_row['patient_id']
        patient_info = patients_pd[patients_pd['patient_id'] == patient_id]
        
        if not patient_info.empty:
            for col in patient_info.columns:
                if col != 'patient_id':  # Avoid duplication
                    record[f'patient_{col}'] = patient_info.iloc[0][col]
        
        # Find EDF file
        file_name = seizure_row['file_name']
        edf_path = None
        
        for root_path in data_root_paths:
            possible_paths = [
                os.path.join(root_path, patient_id, file_name),
                os.path.join(root_path, patient_id.lower(), file_name),
                os.path.join(root_path, patient_id.upper(), file_name),
            ]
            
            for path in possible_paths:
                if os.path.exists(path):
                    edf_path = path
                    break
            
            if edf_path:
                break
        
        if edf_path:
            # Process EDF and extract features - pass the entire row as dict
            edf_features = process_single_edf(edf_path, seizure_row.to_dict())
            record.update(edf_features)
        else:
            record['processing_success'] = False
            record['error_message'] = 'EDF file not found'
        
        all_records.append(record)
    
    # Create comprehensive dataframe using Polars
    comprehensive_df = pl.DataFrame(all_records)
    
    # Print summary
    print("\n" + "="*60)
    print("PROCESSING COMPLETE")
    print("="*60)
    print(f"Total records: {len(comprehensive_df)}")
    
    if 'processing_success' in comprehensive_df.columns:
        success_count = comprehensive_df.filter(pl.col('processing_success')).height
        print(f"Successfully processed: {success_count}")
        print(f"Failed: {len(comprehensive_df) - success_count}")
    
    return comprehensive_df

## Fill in Missing Values

In [16]:
def fill_missing_values_polars(df):
    # Get column types
    str_cols = [col for col in df.columns if df[col].dtype == pl.Utf8]
    num_cols = [col for col in df.columns if df[col].dtype in [pl.Float32, pl.Float64, pl.Int8, pl.Int16, pl.Int32, pl.Int64]]
    
    # Fill string columns with "N/A"
    for col in str_cols:
        df = df.with_columns(pl.col(col).fill_null("N/A"))
    
    # Fill numeric columns with 0
    for col in num_cols:
        df = df.with_columns(
            pl.col(col).fill_null(0).fill_nan(0)
        )
    
    return df

In [17]:
def save_to_csv(df, output_path='comprehensive_df.csv'):
    if isinstance(df, pl.DataFrame):
        df_pd = df.to_pandas()
    else:
        df_pd = df
    
    df_pd.to_csv(output_path, index=False)
    print(f"Dataset saved to: {output_path}")
    print(f"File size: {os.path.getsize(output_path) / 1024 / 1024:.2f} MB")

def save_to_parquet(df, output_path='comprehensive_eeg_features.parquet'):
    if isinstance(df, pl.DataFrame):
        df.write_parquet(output_path)
    else:
        df.to_parquet(output_path, index=False, engine='pyarrow')
    
    print(f"Dataset saved to: {output_path}")
    print(f"File size: {os.path.getsize(output_path) / 1024 / 1024:.2f} MB")

In [18]:
# Load metadata
print("Loading metadata from JSON files...")
seizures_df, patients_df = load_metadata_from_json('data/processing_test_files')
print(f"Loaded {len(patients_df)} patients and {len(seizures_df)} seizures")

Loading metadata from JSON files...
Found 1 JSON files
Loaded 1 patients and 3 seizures


## Main Usage Function

In [19]:
# Build comprehensive dataset
comprehensive_df = build_comprehensive_dataset(
    seizures_df, 
    patients_df,
    data_root_paths=[
        #'data/siena_scalp', 
        #'data/chb-mit',
        'data/processing_test_files'
        ]
)

Processing 3 seizure records...


100%|██████████| 3/3 [36:38<00:00, 732.82s/it]


PROCESSING COMPLETE
Total records: 3
Successfully processed: 3
Failed: 0


In [ ]:
# Fill missing values
#print("\nFilling missing values...")
#comprehensive_df = fill_missing_values_polars(comprehensive_df)

## Export Processed Data

In [ ]:
# Save to Parquet
#print("\nSaving results...")
#save_to_parquet(comprehensive_df, 'processed_data/comprehensive_eeg_features.parquet')

## Load Processed Data

In [ ]:
# Load processed data set
processed_df = pl.read_parquet("processed_data/comprehensive_eeg_features.parquet")

print(f"Shape: {processed_df.shape}")
print(f"Columns: {processed_df.columns}")
